In [252]:
from google.colab import drive
drive.mount('/content/mydrive')

Drive already mounted at /content/mydrive; to attempt to forcibly remount, call drive.mount("/content/mydrive", force_remount=True).


In [253]:
import pandas as pd

train_path = "/content/mydrive/MyDrive/SamSung/Train_samsung.csv"
df = pd.read_csv(train_path)

In [254]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.compose import ColumnTransformer
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier,IsolationForest,GradientBoostingClassifier
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline,make_pipeline
from sklearn.model_selection import cross_val_score
from sklearn.metrics import fbeta_score
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler,LabelEncoder,StandardScaler
from sklearn.tree import DecisionTreeClassifier

In [255]:
from sklearn.pipeline import make_pipeline
cat_pl = make_pipeline(SimpleImputer(
    strategy="most_frequent"))
num_pl = make_pipeline(KNNImputer(n_neighbors=7), StandardScaler())
num_pl


Pipeline(steps=[('knnimputer', KNNImputer(n_neighbors=7)),
                ('standardscaler', StandardScaler())])

In [256]:
from sklearn.pipeline import make_pipeline
cat_pl = make_pipeline(SimpleImputer(
    strategy="most_frequent"), OneHotEncoder())
num_pl = make_pipeline(KNNImputer(n_neighbors=7), MinMaxScaler())
num_pl


Pipeline(steps=[('knnimputer', KNNImputer(n_neighbors=7)),
                ('minmaxscaler', MinMaxScaler())])

In [257]:
num_name = df.select_dtypes(['float', 'int']).columns
num_name
cat_name = df.select_dtypes(['object']).drop(['Class'], axis=1).columns
cat_name


Index(['X1', 'X2', 'X3', 'X4', 'X5', 'X11'], dtype='object')

In [258]:
preprocessor = ColumnTransformer(
    transformers = [
        ('num',num_pl,num_name),
        ('cat',cat_pl,cat_name)
    ]
)
preprocessor

ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('knnimputer',
                                                  KNNImputer(n_neighbors=7)),
                                                 ('minmaxscaler',
                                                  MinMaxScaler())]),
                                 Index(['X6', 'X7', 'X8', 'X9', 'X10'], dtype='object')),
                                ('cat',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehotencoder',
                                                  OneHotEncoder())]),
                                 Index(['X1', 'X2', 'X3', 'X4', 'X5', 'X11'], dtype='object'))])

In [259]:
compelete_pipeline = make_pipeline(preprocessor,RandomForestClassifier())
compelete_pipeline

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('knnimputer',
                                                                   KNNImputer(n_neighbors=7)),
                                                                  ('minmaxscaler',
                                                                   MinMaxScaler())]),
                                                  Index(['X6', 'X7', 'X8', 'X9', 'X10'], dtype='object')),
                                                 ('cat',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehotencoder',
                                                                   OneHotEncoder())]),
                                

In [260]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(sampling_strategy = 'minority')

arr = preprocessor.fit_transform(df)
col_name = ['V'+str(i) for i in range(1,21)]
df_test = pd.DataFrame(arr,columns=col_name)
df_test = pd.concat((df_test,df['Class']),axis=1)
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 491 entries, 0 to 490
Data columns (total 21 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   V1      491 non-null    float64
 1   V2      491 non-null    float64
 2   V3      491 non-null    float64
 3   V4      491 non-null    float64
 4   V5      491 non-null    float64
 5   V6      491 non-null    float64
 6   V7      491 non-null    float64
 7   V8      491 non-null    float64
 8   V9      491 non-null    float64
 9   V10     491 non-null    float64
 10  V11     491 non-null    float64
 11  V12     491 non-null    float64
 12  V13     491 non-null    float64
 13  V14     491 non-null    float64
 14  V15     491 non-null    float64
 15  V16     491 non-null    float64
 16  V17     491 non-null    float64
 17  V18     491 non-null    float64
 18  V19     491 non-null    float64
 19  V20     491 non-null    float64
 20  Class   491 non-null    object 
dtypes: float64(20), object(1)
memory usage:

In [261]:
import lightgbm as lgb
X = df_test.drop('Class', axis=1)
Y = df_test['Class'].astype("category").cat.codes
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.2,random_state=1)
X_test = X_test.to_numpy()
Y_test = Y_test.to_numpy()

X_sm , Y_sm = smote.fit_resample(X_train,Y_train)
X_sm = X_sm.to_numpy()
Y_sm = Y_sm.to_numpy()

models = [LinearSVC(),LogisticRegression(),KNeighborsClassifier(n_neighbors=7),GaussianNB(),DecisionTreeClassifier(),GradientBoostingClassifier(n_estimators=200,max_depth=10),MLPClassifier()]


In [262]:
from sklearn.metrics import classification_report

for model in models:
    model.fit(X_sm,Y_sm)
    y_pred = model.predict(X_test)
    print(model)
    print(classification_report(Y_test,y_pred))

LinearSVC()
              precision    recall  f1-score   support

           0       0.62      0.75      0.68        32
           1       0.87      0.78      0.82        67

    accuracy                           0.77        99
   macro avg       0.74      0.76      0.75        99
weighted avg       0.79      0.77      0.77        99

LogisticRegression()
              precision    recall  f1-score   support

           0       0.59      0.75      0.66        32
           1       0.86      0.75      0.80        67

    accuracy                           0.75        99
   macro avg       0.72      0.75      0.73        99
weighted avg       0.77      0.75      0.75        99

KNeighborsClassifier(n_neighbors=7)
              precision    recall  f1-score   support

           0       0.41      0.47      0.43        32
           1       0.73      0.67      0.70        67

    accuracy                           0.61        99
   macro avg       0.57      0.57      0.57        99
weigh

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  ConvergenceWarning,


In [263]:
from sklearn.model_selection import GridSearchCV

prameter = {
    'n_estimators' : [100,200,300],
}
clf = GridSearchCV(GradientBoostingClassifier(),prameter)
clf.fit(X_train,Y_train)

GridSearchCV(estimator=GradientBoostingClassifier(),
             param_grid={'n_estimators': [100, 200, 300]})

In [264]:
clf.best_params_

{'n_estimators': 100}

In [265]:
print(np.sum(Y_test))


67


In [267]:
from sklearn.metrics import f1_score
model = GradientBoostingClassifier(n_estimators=100)
model.fit(X_train,Y_train)
y_pred=  model.predict(X_test)
print(classification_report(Y_test,y_pred))

              precision    recall  f1-score   support

           0       0.78      0.56      0.65        32
           1       0.82      0.93      0.87        67

    accuracy                           0.81        99
   macro avg       0.80      0.74      0.76        99
weighted avg       0.81      0.81      0.80        99



/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but GradientBoostingClassifier was fitted with feature names
  "X does not have valid feature names, but"
